# Add fits files with same name from 2 different folders 

In [58]:
import os

from IPython.html import widgets
from IPython.display import display

import matplotlib.pyplot as plt

import glob
import pyfits

In [59]:
%gui qt

In [60]:
root_folder = '/Users/j35/Desktop/'

In [61]:
from PyQt4 import QtCore, QtGui

def gui_fname(dir=None):
    """Select a file via a dialog and returns the file name.
    """
    if dir is None: dir ='./'
    dir_name = QtGui.QFileDialog.getExistingDirectory(None, "Select Folder ...",
                                                  dir,
                                                  QtGui.QFileDialog.ShowDirsOnly)
    return dir_name

### Select folder1 

In [62]:
%gui qt
folder_1 = gui_fname(dir = root_folder)
%matplotlib inline

### Select folder2

In [63]:
%gui qt
folder_2 = gui_fname(dir = root_folder)
%matplotlib inline

### Check input 

In [64]:
list_fname_1 = glob.glob(folder_1 + '/*.fits')
list_fname_2 = glob.glob(folder_2 + '/*.fits')
assert len(list_fname_1) == len(list_fname_2)

## loading the data 

In [65]:
w = widgets.IntProgress()
w.max = len(list_fname_1)-1
display(w)

index = 0
data_folder_1 = []
data_folder_2 = []
for index in range(len(list_fname_1)):
    hdu_list = pyfits.open(list_fname_1[index])
    hdu = hdu_list[0]
    data_folder_1.append(hdu.data)

    hdu_list = pyfits.open(list_fname_2[index])
    hud = hdu_list[0]
    data_folder_2.append(hdu.data)
    
    w.value = index

In [66]:
def digit_file_name(full_file_name):
    base_name = os.path.basename(full_file_name)
    [short_name, ext] = base_name.split('.')
    [image_name, image_number] = short_name.split('_')
    return [image_name, image_number]

In [67]:
w = widgets.IntProgress()
w.max = len(list_fname_1)-1
display(w)

sum_data = []
sum_names = []
for index in range(len(list_fname_1)):
    
    [image_name_1, image_number_1] = digit_file_name(list_fname_1[index])
    [image_name_2, image_number_2] = digit_file_name(list_fname_2[index])
    
    #make sure that files to add have same name
    if image_number_1 == image_number_2:
        _new_data = data_folder_1[index] + data_folder_2[index]
        sum_data.append(_new_data)
        _new_name = image_name_1 + '_add_' + image_name_2 + '_' + image_number_1 + '.fits'
        sum_names.append(_new_name)
    else:
        raise ValueError("Files to add do not have the same name!")
        
    w.value = index    

## Exporting data 

In [68]:
from PyQt4 import QtCore, QtGui

def gui_fname(dir=None):
    """Select a folder
    """
    if dir is None: dir ='./'
    dir_name = QtGui.QFileDialog.getExistingDirectory(None, "Select Folder ...",
                                                  dir,
                                                  QtGui.QFileDialog.ShowDirsOnly)
    return dir_name

In [69]:
%gui qt
dir_name = gui_fname(dir = root_folder)
%matplotlib inline

In [70]:
w = widgets.IntProgress()
w.max = len(list_fname_1)-1
display(w)

for index in range(len(list_fname_1)):
    
    full_name = os.path.join(dir_name, sum_names[index])
    
    hdu = pyfits.PrimaryHDU(sum_data[index])
    hdulist = pyfits.HDUList([hdu])
    hdulist.writeto(full_name)
    
    w.value = index